In [2]:
import numpy as np
import pandas as pd


In [5]:
df1 = pd.read_excel(r'C:\Users\dneupane\Documents\pfra-try\notebooks\pluvial\c_df.xlsx')
df1 = df1.set_index('TR')

#select events to prepare nested distribution
Return_Intervals = np.array([2, 5, 10, 25, 50, 100, 200, 500, 1000])  # Return intervals for calculating runoff values.

hyetos = {}
for evnt in Return_Intervals:
    df = df1.loc[evnt][['duration', 'value']].set_index('duration')

df['duration_min'] = [5, 10, 15, 30, 60, 120, 180, 360, 720, 1440]
df['duration_hr'] = df['duration_min']/60
df['log_dur'] = np.log10(df['duration_hr'])
df['log_dep'] = np.log10(df['value'])
poly = np.polyfit(df['log_dur'], df['log_dep'], 2)  #fitting a polynomial equation for log of depth and duration
a, b, c = poly   #three coefficients of the fitted polynomial equation

#Inputs for rainfall
td = 12 #storm duration
tp = 3 #time to peak
t_step = 0.1 #time step
P_max = df.at['12h', 'value']  #maximum rainfall value at 12 hour 

ratio_to_12h = pd.DataFrame(np.arange(start=0, stop=12.1, step=0.1), columns = ['time'])
t = ratio_to_12h['time']

#Calculation of duration
for i in range(120):
    if t[i] < tp:
        x = (tp - t[i])*(td/tp)
    elif t[i] > tp:
        x = ((t[i] - tp)*td)/(td-tp)
    else: x = t[i]
    ratio_to_12h.loc[i, 'duration'] = x
ratio_to_12h.loc[120, 'duration'] = 12

#Calculation of depth
for i in range(121):
    ratio_to_12h.loc[i, 'depth'] = pow(10, (a * np.square(np.log10(ratio_to_12h.loc[i, 'duration'])) + b * np.log10(ratio_to_12h.loc[i, 'duration']) + c))

#Calculation of cumulative depth at each time step
for i in range(1,121):
    if t[i] < tp:
        y = (tp/td)*(P_max - ratio_to_12h.loc[i, 'depth'])
    elif t[i] > tp:
        y = (tp/td)*P_max + (1-(tp/td)) * ratio_to_12h.loc[i, 'depth']
    ratio_to_12h.loc[i, 'cum_depth'] = y
ratio_to_12h.loc[0, 'cum_depth'] = 0

#Caculation of ratio and slope
ratio_slope = pd.DataFrame(np.arange(start=0, stop=12.1, step=0.1), columns= ['time'])
ratio_slope['ratio'] = ratio_to_12h['cum_depth']/ratio_to_12h.loc[120, 'cum_depth'] #ratio calculation
ratio_slope['slope'] = ratio_slope['ratio'].diff()/0.1 #slope calculation
ratio_slope.loc[0, 'slope'] = 0
ratio_slope['t_step'] = ratio_slope['time']
ratio_slope = ratio_slope.set_index(['time'])

In [6]:
ratio_slope

,ratio,slope,t_step
time,,,
0.0,0.000000,0.000000,0.0
0.1,0.003157,0.031571,0.1
0.2,0.005042,0.018845,0.2
0.3,0.007005,0.019631,0.3
0.4,0.009052,0.020476,0.4
...,...,...,...
11.6,0.992708,0.018843,11.6
11.7,0.994567,0.018593,11.7
11.8,0.996402,0.018348,11.8


In [60]:
df1 = pd.read_excel(r'C:\Users\dneupane\Documents\pfra-try\notebooks\pluvial\c_df.xlsx')
df1 = df1.set_index('TR')

#select events to prepare nested distribution
Return_Intervals = np.array([2, 5, 10, 25, 50, 100, 200, 500, 1000])  # Return intervals for calculating runoff values.

hyetos = {}
for evnt in Return_Intervals:
    df = df1.loc[evnt][['duration', 'value']].set_index('duration')

df['duration_min'] = [5, 10, 15, 30, 60, 120, 180, 360, 720, 1440]
df['duration_hr'] = df['duration_min']/60
df['log_dur'] = np.log10(df['duration_hr'])
df['log_dep'] = np.log10(df['value'])
poly = np.polyfit(df['log_dur'], df['log_dep'], 2)  #fitting a polynomial equation for log of depth and duration
a, b, c = poly   #three coefficients of the fitted polynomial equation

#Inputs for rainfall
td = 12 #storm duration
tp = 3 #time to peak
t_step = 0.1 #time step
P_max = df.at['12h', 'value']  #maximum rainfall value at 12 hour 

ratio_to_12h = pd.DataFrame(np.arange(start=0, stop=12.1, step=0.1), columns = ['time'])
t = ratio_to_12h['time']

#Calculation of duration
for i in range(120):
    if t[i] < tp:
        x = (tp - t[i])*(td/tp)
    else:
        x = ((t[i] - tp)*td)/(td-tp)
    ratio_to_12h.loc[i, 'duration'] = x
ratio_to_12h.loc[120, 'duration'] = 12

#Calculation of depth
for i in range(121):
    ratio_to_12h.loc[i, 'depth'] = pow(10, (a * np.square(np.log10(ratio_to_12h.loc[i, 'duration'])) + b * np.log10(ratio_to_12h.loc[i, 'duration']) + c))

#Calculation of cumulative depth at each time step
for i in range(1,121):
    if t[i] < tp:
        y = (tp/td)*(P_max - ratio_to_12h.loc[i, 'depth'])
    else:
        y = (tp/td)*P_max + (1-(tp/td)) * ratio_to_12h.loc[i, 'depth']
    ratio_to_12h.loc[i, 'cum_depth'] = y
ratio_to_12h.loc[0, 'cum_depth'] = 0

#Caculation of ratio and slope
ratio_slope = pd.DataFrame(np.arange(start=0, stop=12.1, step=0.1), columns= ['time'])
ratio_slope['ratio'] = ratio_to_12h['cum_depth']/ratio_to_12h.loc[120, 'cum_depth'] #ratio calculation
ratio_slope['slope'] = ratio_slope['ratio'].diff() #slope calculation
ratio_slope.loc[0, 'slope'] = 0
ratio_slope['t_step'] = ratio_slope['time']
ratio_slope.set_index(['time'])

C:\Users\dneupane\AppData\Local\Temp\ipykernel_25928\1940803490.py:38: RuntimeWarning: divide by zero encountered in log10
  ratio_to_12h.loc[i, 'depth'] = pow(10, (a * np.square(np.log10(ratio_to_12h.loc[i, 'duration'])) + b * np.log10(ratio_to_12h.loc[i, 'duration']) + c))


,ratio,slope,t_step
time,,,
0.0,0.000000,0.000000,0.0
0.1,0.003157,0.003157,0.1
0.2,0.005042,0.001884,0.2
0.3,0.007005,0.001963,0.3
0.4,0.009052,0.002048,0.4
...,...,...,...
11.6,0.992708,0.001884,11.6
11.7,0.994567,0.001859,11.7
11.8,0.996402,0.001835,11.8


In [61]:
ratio_slope.loc[30]

time      3.000000
ratio     0.251010
slope     0.076243
t_step    3.000000
Name: 30, dtype: float64

In [3]:
df1 = pd.read_excel(r'C:\Users\dneupane\Documents\pfra-try\notebooks\pluvial\c_df.xlsx')
df1 = df1.set_index('TR')

In [4]:
#select events to prepare nested distribution
Return_Intervals = np.array([2, 5, 10, 25, 50, 100])  # Return intervals for calculating runoff values.

In [5]:
hyetos = {}
for evnt in Return_Intervals:
    df = df1.loc[evnt][['duration', 'value']].set_index('duration')

In [6]:
df['duration_min'] = [5, 10, 15, 30, 60, 120, 180, 360, 720, 1440]
df['duration_hr'] = df['duration_min']/60
 
df['log_dur'] = np.log10(df['duration_hr'])
df['log_dep'] = np.log10(df['value'])

poly = np.polyfit(df['log_dur'], df['log_dep'], 2)  #fitting a polynomial equation for log of depth and duration
a, b, c = poly   #three coefficients of the fitted polynomial equation

In [46]:
td = 12 #storm duration
tp = 3 #time to peak
t_step = 0.1 #time step
P_max = df.at['12h', 'value']  #maximum rainfall value at 12 hour 

ratio_to_12h = pd.DataFrame(np.arange(start=0, stop=12.1, step=0.1), columns = ['time'])
t = ratio_to_12h['time']

In [47]:
#Calculation of duration
for i in range(120):
    if t[i] < tp:
        x = (tp - t[i])*(td/tp)
    else:
        x = ((t[i] - tp)*td)/(td-tp)
    ratio_to_12h.loc[i, 'duration'] = x
ratio_to_12h.loc[120, 'duration'] = 12


In [48]:
#Calculation of depth
for i in range(121):
    ratio_to_12h.loc[i, 'depth'] = pow(10, (a * np.square(np.log10(ratio_to_12h.loc[i, 'duration'])) + b * np.log10(ratio_to_12h.loc[i, 'duration']) + c))

C:\Users\dneupane\AppData\Local\Temp\ipykernel_25928\71889310.py:3: RuntimeWarning: divide by zero encountered in log10
  ratio_to_12h.loc[i, 'depth'] = pow(10, (a * np.square(np.log10(ratio_to_12h.loc[i, 'duration'])) + b * np.log10(ratio_to_12h.loc[i, 'duration']) + c))


In [49]:
np.log10(1)

0.0

In [50]:
#Calculation of cumulative depth at each time step
for i in range(1,121):
    if t[i] < tp:
        y = (tp/td)*(P_max - ratio_to_12h.loc[i, 'depth'])
    else:
        y = (tp/td)*P_max + (1-(tp/td)) * ratio_to_12h.loc[i, 'depth']
    ratio_to_12h.loc[i, 'cum_depth'] = y
ratio_to_12h.loc[0, 'cum_depth'] = 0
ratio_to_12h

,time,duration,depth,cum_depth
0,0.0,12.000000,7.752556,0.000000
1,0.1,11.600000,7.697044,0.023244
2,0.2,11.200000,7.639295,0.037682
3,0.3,10.800000,7.579150,0.052718
4,0.4,10.400000,7.516431,0.068398
...,...,...,...,...
116,11.6,11.466667,7.678050,7.706043
117,11.7,11.600000,7.697044,7.720288
118,11.8,11.733333,7.715789,7.734347
119,11.9,11.866667,7.734291,7.748224


In [51]:

ratio_to_12h.loc[29]

time         2.900000
duration     0.400000
depth        2.433134
cum_depth    1.339222
Name: 29, dtype: float64

In [52]:
ratio_to_12h.loc[30]

time         3.000000
duration     0.000000
depth        0.000000
cum_depth    1.947505
Name: 30, dtype: float64

In [53]:
#Caculation of ratio and slope
ratio_slope = pd.DataFrame(np.arange(start=0, stop=12.1, step=0.1), columns= ['time'])
for i in range(121):
    ratio_slope.loc[i, 'ratio'] = ratio_to_12h.loc[i, 'cum_depth']/ratio_to_12h.loc[120, 'cum_depth']
ratio_slope['slope'] = ratio_slope['ratio'].diff()/0.1
ratio_slope.loc[0, 'slope'] = 0
#ratio_to_12h = pd.DataFrame(np.arange(start=0, stop=12.1, step=0.1), columns = ['time'])


In [54]:
ratio_slope = pd.DataFrame(np.arange(start=0, stop=12.1, step=0.1), columns= ['time'])
ratio_slope['ratio'] = ratio_to_12h['cum_depth']/ratio_to_12h.loc[120, 'cum_depth'] #ratio calculation
ratio_slope['slope'] = ratio_slope['ratio'].diff()/0.1 #slope calculation
ratio_slope.loc[0, 'slope'] = 0
ratio_slope['t_step'] = ratio_slope['time']
ratio_slope.set_index(['time'])

,ratio,slope,t_step
time,,,
0.0,0.000000,0.000000,0.0
0.1,0.002995,0.029947,0.1
0.2,0.004855,0.018600,0.2
0.3,0.006792,0.019372,0.3
0.4,0.008812,0.020201,0.4
...,...,...,...
11.6,0.992801,0.018598,11.6
11.7,0.994636,0.018353,11.7
11.8,0.996447,0.018113,11.8


In [55]:
ratio_slope.loc[30]

time      3.000000
ratio     0.250905
slope     0.783676
t_step    3.000000
Name: 30, dtype: float64

In [56]:
ratio_slope.loc[29]

time      2.900000
ratio     0.172537
slope     0.282601
t_step    2.900000
Name: 29, dtype: float64